In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
B, T, C = 2, 3, 2  # Batch size = 2, Time steps = 3, Feature size = 2

# Example tensor `x`
x = torch.tensor([
    [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]],    # Batch 1
    [[7.0, 8.0], [9.0, 10.0], [11.0, 12.0]]  # Batch 2
])

print("Input tensor `x`:")
print(x)

Input tensor `x`:
tensor([[[ 1.,  2.],
         [ 3.,  4.],
         [ 5.,  6.]],

        [[ 7.,  8.],
         [ 9., 10.],
         [11., 12.]]])


In [ ]:
# bag of words
xbow = torch.zeros((B,T,C))
print("xbow:")
print(xbow)
for b in range(B):
    for t in range(T):
            xprev = x[b,:t+1]
            print("xprev:")
            print(xprev)
            xbow[b,t] = torch.mean(xprev,0)
            print("xbow:")
            print(xbow)

In [ ]:
cumulative_sum = torch.cumsum(x, dim=1)
print("cumulative_sum:")
print(cumulative_sum)
time_indices = torch.arange(1, T+1, device=x.device).view(1, T, 1)
print("time_indices:")
print(time_indices)
xbow2 = cumulative_sum / time_indices
print("xbow2:")
print(xbow2)


In [34]:
T, C = 8, 32
x = torch.randn(T, C)

head_size = 8
query = nn.Linear(C, head_size)
key = nn.Linear(C, head_size)
value = nn.Linear(C, head_size)

q = query(x)
k = key(x)

wei = q @ k.transpose(-2, -1)

tril = torch.tril(torch.ones(T, T), diagonal=0)

wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v





In [ ]:
print("wei:")
print(wei)

In [ ]:
print("tril:")
print(tril)

In [35]:
print("out:")
print(out)

out:
tensor([[ 0.0649,  0.1602, -0.0311,  0.6902, -0.7329, -0.5998, -0.5210, -0.2716],
        [-0.0886, -0.0320, -0.0311,  0.7142, -0.2816,  0.0311, -0.1482, -0.1849],
        [-0.2431,  0.2085, -0.0192,  0.6746, -0.3607,  0.3686, -0.2317, -0.3600],
        [-0.2869, -0.1572, -0.0683,  0.7051,  0.1820,  0.8295,  0.2040, -0.0929],
        [ 0.1469,  0.0730, -0.2744,  0.5075, -0.3719, -0.0090, -0.4512,  0.1519],
        [-0.0554,  0.3178, -0.1247,  0.4949, -0.1455,  0.5526, -0.1992, -0.0362],
        [-0.0352,  0.1734, -0.1119,  0.5989, -0.1932,  0.3782, -0.2868,  0.0249],
        [-0.3431,  0.4761, -0.1086,  0.6183, -0.0982,  0.5465,  0.0169, -0.0715]],
       grad_fn=<MmBackward0>)
